# Maps

In Spark, maps take data as input and then transform that data with whatever function you put in the map. They are like directions for the data telling how each input should get to the output.

The first code cell creates a SparkContext object. With the SparkContext, you can input a dataset and parallelize the data across a cluster (since you are currently using Spark in local mode on a single machine, technically the dataset isn't distributed yet).

Run the code cell below to instantiate a SparkContext object and then read in the log_of_songs list into Spark. 

在 Spark 中，map会把输入数据按照map中的函数进行转换它们就像数据的指引，告诉每个输入要如何到达输出 。

第一个单元创建一个 SparkContext 对象。你可以使用 SparkContext 把输入数据分布到集群中（因为你当前使用的是单机模式，严格讲数据集没有分布到集群上）。

运行下面的代码单元来实例化 SparkContext 对象，然后用Spark 读取 log_of_songs 列表。

In [1]:
### 
# You might have noticed this code in the screencast.
#
# import findspark
# findspark.init('spark-2.3.2-bin-hadoop2.7')
#
# The findspark Python module makes it easier to install
# Spark in local mode on your computer. This is convenient
# for practicing Spark syntax locally. 
# However, the workspaces already have Spark installed and you do not
# need to use the findspark module
#
###

import pyspark
sc = pyspark.SparkContext(appName="maps_and_lazy_evaluation_example")

log_of_songs = [
        "Despacito",
        "Nice for what",
        "No tears left to cry",
        "Despacito",
        "Havana",
        "In my feelings",
        "Nice for what",
        "despacito",
        "All the stars"
]

# parallelize the log_of_songs to use with Spark
distributed_song_log = sc.parallelize(log_of_songs)

This next code cell defines a function that converts a song title to lowercase. Then there is an example converting the word "Havana" to "havana".

下一个代码单元定义了一个将歌曲标题转换为小写的函数。有一个例子将 “Havana"  改写成  “havana"。

In [2]:
def convert_song_to_lowercase(song):
    return song.lower()

convert_song_to_lowercase("Havana")

'havana'

The following code cells demonstrate how to apply this function using a map step. The map step will go through each song in the list and apply the convert_song_to_lowercase() function. 

以下代码单元演示了如何通过 map 来应用此函数。map 步骤会遍历列表中的每首歌曲并且把convert_song_to_lowercase() 函数应用在歌曲上。

In [4]:
distributed_song_log.map(convert_song_to_lowercase)

PythonRDD[1] at RDD at PythonRDD.scala:53

You'll notice that this code cell ran quite quickly. This is because of lazy evaluation. Spark does not actually execute the map step unless it needs to.

"RDD" in the output refers to resilient distributed dataset. RDDs are exactly what they say they are: fault-tolerant datasets distributed across a cluster. This is how Spark stores data. 

To get Spark to actually run the map step, you need to use an "action". One available action is the collect method. The collect() method takes the results from all of the clusters and "collects" them into a single list on the master node.

这个代码单元运行得很快。这是惰性评估的原因。不必要的情况下 Spark 不会执行 map 步骤。

输出中的 “RDD” 指的是弹性分布式数据集。RDD 正是他们所说的：分布在集群中的容错数据集。这就是 Spark 存储数据的方式。 

要让 Spark 运行 map 步骤，你需要使用一个“工具”。有个可用的“工具”叫 collect 方法。collect() 方法会把所有集群中结果收集到主节点上的单个列表中。

**基于MacJava8环境切换:**
1. java版本切换
https://www.java.com/zh_CN/download/ 官网下载 apk
201910月看到的是 Version8 Update221 (20190716)
2. 检查版本
```
$ /usr/libexec/java_home -V
Matching Java Virtual Machines (2):
    12.0.1, x86_64:	"Java SE 12.0.1"	/Library/Java/JavaVirtualMachines/jdk-12.0.1.jdk/Contents/Home
    1.8.0_212, x86_64:	"Java SE 8"	/Library/Java/JavaVirtualMachines/jdk1.8.0_212.jdk/Contents/Home
```
3. 切换版本
```
$ export JAVA_HOME=`/usr/libexec/java_home -v 1.8.0_212`
```
4. 检查版本
```
$ java -version
java version "1.8.0_212"
Java(TM) SE Runtime Environment (build 1.8.0_212-b10)
Java HotSpot(TM) 64-Bit Server VM (build 25.212-b10, mixed mode)
```
5. 注意事项
- 貌似默认还是java12的, 需要在一个终端中做完上述操作后,再运行conda环境 和 notebook
- https://kodejava.org/how-do-i-set-the-default-java-jdk-version-on-mac-os-x/

In [8]:
distributed_song_log.map(convert_song_to_lowercase).collect()
# java version 不是8的话会报错,需要降级

['despacito',
 'nice for what',
 'no tears left to cry',
 'despacito',
 'havana',
 'in my feelings',
 'nice for what',
 'despacito',
 'all the stars']

In [11]:
# spark 和 python 的时间处理
## 上述代码第一次run 2.7s, 后续为92ms
## 这可能和 spark 开始对数据的初始化工作有关
## 对比下纯 python 的方法 10ms:
## 感觉初始化之后的时间spark很慢, 可能和数据集大小有关
new_list = []
for song in log_of_songs:
    new_list.append(song.lower())
    
new_list

['despacito',
 'nice for what',
 'no tears left to cry',
 'despacito',
 'havana',
 'in my feelings',
 'nice for what',
 'despacito',
 'all the stars']

Note as well that Spark is not changing the original data set: Spark is merely making a copy. You can see this by running collect() on the original dataset.

另外，请注意，Spark 不会更改原始数据集：Spark 只是复制了一份副本。你可以在原始数据集上运行collect() 来确认这点。

In [12]:
distributed_song_log.collect()

['Despacito',
 'Nice for what',
 'No tears left to cry',
 'Despacito',
 'Havana',
 'In my feelings',
 'Nice for what',
 'despacito',
 'All the stars']

You do not always have to write a custom function for the map step. You can also use anonymous (lambda) functions as well as built-in Python functions like string.lower(). 

Anonymous functions are actually a Python feature for writing functional style programs.

在 map 步骤，有时你不用编写自定义函数。你还可以使用匿名（lambda）函数以及string.lower() 等内置Python函数。 

匿名函数实际上是一个Python特性，专门用于编写函数式程序。

In [15]:
distributed_song_log.map(lambda x: x.lower()).collect()

['despacito',
 'nice for what',
 'no tears left to cry',
 'despacito',
 'havana',
 'in my feelings',
 'nice for what',
 'despacito',
 'all the stars']